# Optimized Turnbull Driver

In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from matplotlib.backends.backend_pdf import PdfPages

from scipy.stats import truncnorm

In [2]:
%run ../source/models/iceberg/iceberg.py
%run ../source/models/metocean/metocean.py
%run ../source/models/drift/turnbull.py
%run ../source/plotting/plot.py

In [3]:
chosen_track_ind = 0
iip_berg_df, iip_df, chosen_inds_arr = get_berg_df(2015, chosen_track_ind)
iip_berg_df

,index,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,TIMESTAMP
0,6319,2015,21511,6/16/2015,1442,48.16,-50.01,R/V,LG,TAB,GTJZ,2015-06-16 14:42:00
1,6320,2015,21511,6/17/2015,1300,48.09,-50.02,R/V,LG,GEN,GMRS,2015-06-17 13:00:00
2,6321,2015,21511,6/18/2015,1249,48.08,-49.76,VIS,LG,TAB,GTJZ,2015-06-18 12:49:00


In [4]:
chosen_inds_arr.sort(key=len, reverse=True)
iip_berg_df = iip_df.loc[chosen_inds_arr[0]].reset_index()
iip_berg_df

,index,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,TIMESTAMP
0,10822,2015,23525,6/24/2015,1306,47.71,-49.50,R/V,LG,GEN,GTJZ,2015-06-24 13:06:00
1,10823,2015,23525,6/25/2015,1118,47.55,-49.41,R/V,LG,GEN,GTJZ,2015-06-25 11:18:00
2,10824,2015,23525,6/25/2015,1918,47.49,-49.44,R/V,LG,GEN,GPGR,2015-06-25 19:18:00
3,10825,2015,23525,6/26/2015,1046,47.46,-49.45,R/V,GEN,GEN,GTJZ,2015-06-26 10:46:00
4,10826,2015,23525,6/26/2015,1636,47.43,-49.44,R/V,LG,GEN,GMRS,2015-06-26 16:36:00
5,10827,2015,23525,6/27/2015,1105,47.38,-49.47,R/V,LG,GEN,GPGR,2015-06-27 11:05:00
6,10828,2015,23525,6/27/2015,1331,47.35,-49.49,RAD,GEN,GEN,2005,2015-06-27 13:31:00


In [5]:
iip_berg = Iceberg(iip_berg_df['ICEBERG_NUMBER'].loc[0],
                    iip_berg_df['TIMESTAMP'].dt.to_pydatetime(),
                    [0], [0],
                    iip_berg_df['SIGHTING_LATITUDE'].loc[:].tolist(),
                    iip_berg_df['SIGHTING_LONGITUDE'].loc[:].tolist(),
                    iip_berg_df['SIZE'].loc[0],
                    iip_berg_df['SHAPE'].loc[0])

In [6]:
ocean_data = ECMWF_Ocean(min(iip_berg.lons), max(iip_berg.lons),
                         min(iip_berg.lats), max(iip_berg.lats),
                         iip_berg.datetimes[0], iip_berg.datetimes[-1])

In [7]:
atm_data = ECMWF_Atm(min(iip_berg.lons), max(iip_berg.lons),
                     min(iip_berg.lats), max(iip_berg.lats),
                     iip_berg.datetimes[0], iip_berg.datetimes[-1])

In [63]:
def optimize_turnbull(Ca):
    
    Cw = 2.25

    # Buffer number for bounding spatial box
    buf = 0.5


    # Create model iceberg
    mod_berg = Iceberg(iip_berg_df['ICEBERG_NUMBER'][0],
                        [iip_berg_df['TIMESTAMP'].dt.to_pydatetime()[0]],
                        [0], [0],
                        [iip_berg_df['SIGHTING_LATITUDE'][0]],
                        [iip_berg_df['SIGHTING_LONGITUDE'][0]],
                        iip_berg_df['SIZE'][0], iip_berg_df['SHAPE'].loc[0])

    mod_berg.length = [mod_berg.length]
    mod_berg.width = [mod_berg.width]
    mod_berg.height = [mod_berg.height]
    
    mod_berg.air_drag_coeff = Ca
    mod_berg.water_drag_coeff = Cw
    

    # Timesteps
    t0 = iip_berg.datetimes[0]
    tf = iip_berg.datetimes[-1]
    t_delta = tf - t0
    t_delta_hours = t_delta.days*24 + t_delta.seconds/3600
    t_step_hours = 0.1
    t_step = timedelta(hours = t_step_hours)
    dt = t_step_hours*3600  # timestep in seconds
    t_all = []
    for i in np.arange(0, t_delta_hours + t_step_hours, t_step_hours):
        new_date = t0 + timedelta(hours=i)
        t_all.append(new_date)



    # Iceberg Initials
    x0, y0 = mod_berg.lons[0], mod_berg.lats[0]
    l0, w0, h0 = mod_berg.length[0], mod_berg.width[0], mod_berg.height[0]
    vx0 = 0
    vy0 = 0

    l, w, h = l0, w0, h0
    x, y = x0, y0
    vx, vy = vx0, vy0


    for t in t_all:


        t_ocean = nc.date2num(t, 'hours since 1950-01-01 00:00:00.0 00:00', 'standard')
        t_atm = nc.date2num(t, 'hours since 1900-01-01 00:00:00.0 00:00', 'standard')

        vau = atm_data.iUA([t_atm, y, x])[0]
        vav = atm_data.iVA([t_atm, y, x])[0]
        vwu = ocean_data.iUW([t_ocean, y, x])[0]
        vwv = ocean_data.iVW([t_ocean, y, x])[0]


        vx, vy, x, y = turnbull_drift(mod_berg, vau, vav, vwu, vwv, dt) 


        if x > max(iip_berg.lons)+buf or x < min(iip_berg.lons)-buf or y > max(iip_berg.lats)+buf or y <= min(iip_berg.lats)-buf:
            # Iceberg out-of-bounds
            print('out-of-bounds')
            break


        else:
            mod_berg.xvels.append(vx)
            mod_berg.yvels.append(vy)
            mod_berg.lons.append(x)
            mod_berg.lats.append(y)
            mod_berg.datetimes.append(t)

            
    all_iip_times = []
    for iip_datetime in iip_berg.datetimes:
        all_iip_times.append(round((iip_datetime - iip_berg.datetimes[0]).total_seconds()/3600,1))
    all_iip_times = np.asarray(all_iip_times)
    
    all_mod_times = []
    for mod_datetime in mod_berg.datetimes:
        all_mod_times.append(round((mod_datetime - mod_berg.datetimes[0]).total_seconds()/3600,1))
    all_mod_times = np.asarray(all_mod_times)

    time_inds = []
    for j in range(all_iip_times.size):
        if np.any(all_mod_times == all_iip_times[j]):
            time_inds.append(np.where(all_mod_times == all_iip_times[j])[0][0])
                        
    all_xy_mse = []
    counter = 0
    for iip_time_ind, mod_time_ind in enumerate(time_inds):
        xy_mse = np.sqrt((mod_berg.lons[mod_time_ind] - iip_berg.lons[iip_time_ind])**2 + (mod_berg.lats[mod_time_ind] - iip_berg.lats[iip_time_ind])**2)
        all_xy_mse.append(xy_mse)
       
    avg_xy_mse = np.mean(all_xy_mse)
    
    return avg_xy_mse

In [42]:
from scipy.optimize import minimize

In [64]:
optimize_turnbull(0.25)

0.12020411633621961

In [65]:
minimize(optimize_turnbull, x0=0.25)

      fun: 0.12285889098228649
 hess_inv: array([[  3.39614691e-12]])
      jac: array([ 15765.50372943])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 88
      nit: 1
     njev: 26
   status: 2
  success: False
        x: array([ 0.249999])